In [ ]:
!pip install pdfminer.six

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 39.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import requests
import os
from pdfminer.high_level import extract_text

In [ ]:
# Load the Excel file
file_path = '/content/drive/MyDrive/DK/Stateside/Model_Finetunig/Datasets/cleaned_excel_1500_Completed.xlsx'
df = pd.read_excel(file_path)

# Display the first few rows of the dataframe to understand its structure
df.head()

,StateLinkID,BillNumber,BillState,BillTextURL,BillYear,Summary_StateLink
0,262759,2 2024,BRA,https://static.poder360.com.br/2024/07/NOTA-TE...,2024,Compliance with Advertising Transparency Stand...
1,262575,HB 4910,MA,https://malegislature.gov/Bills/193/H4910.pdf,2024,This measure relates to pharmaceutical manufac...
2,262533,41 2024,IRL,https://data.oireachtas.ie/ie/oireachtas/bill/...,2024,The bill aims to provide substantial financial...
3,262532,42 2024,IRL,https://data.oireachtas.ie/ie/oireachtas/bill/...,2024,The bill amends the Electricity Regulation Act...
4,262531,24 244,DEU,https://dserver.bundestag.de/btd/20/118/201185...,2024,The bill aims to enhance the effectiveness and...


In [ ]:
# Drop rows with missing URLs or summaries
df.dropna(subset=['BillTextURL', 'Summary_StateLink'], inplace=True)

# Fill any other missing cells with a placeholder (e.g., an empty string)
df.fillna('', inplace=True)

# Check for any duplicates and remove them if necessary
df.drop_duplicates(inplace=True)

# Strip any leading or trailing spaces in the 'BillTextURL' and 'Summary' columns
df['BillTextURL'] = df['BillTextURL'].str.strip()
df['Summary_StateLink'] = df['Summary_StateLink'].str.strip()

# Display the cleaned dataframe
df.head()

,StateLinkID,BillNumber,BillState,BillTextURL,BillYear,Summary_StateLink
0,262759,2 2024,BRA,https://static.poder360.com.br/2024/07/NOTA-TE...,2024,Compliance with Advertising Transparency Stand...
1,262575,HB 4910,MA,https://malegislature.gov/Bills/193/H4910.pdf,2024,This measure relates to pharmaceutical manufac...
2,262533,41 2024,IRL,https://data.oireachtas.ie/ie/oireachtas/bill/...,2024,The bill aims to provide substantial financial...
3,262532,42 2024,IRL,https://data.oireachtas.ie/ie/oireachtas/bill/...,2024,The bill amends the Electricity Regulation Act...
4,262531,24 244,DEU,https://dserver.bundestag.de/btd/20/118/201185...,2024,The bill aims to enhance the effectiveness and...


In [ ]:
pdf_dir = '/content/drive/MyDrive/DK/Stateside/Model_Finetunig/Training1500GPU'
os.makedirs(pdf_dir, exist_ok=True)

In [ ]:
def download_pdf(url, save_path):
    response = requests.get(url)
    with open(save_path, 'wb') as f:
        f.write(response.content)

def extract_pdf_text(pdf_path):
    return extract_text(pdf_path)

In [ ]:
# Apply the download and extract process
df['BillText'] = ''

for index, row in df.iterrows():
    pdf_url = row['BillTextURL']
    pdf_path = os.path.join(pdf_dir, f'bill_{index}.pdf')

    try:
        # Download the PDF
        download_pdf(pdf_url, pdf_path)

        # Extract text from the PDF
        text = extract_pdf_text(pdf_path)
        df.at[index, 'BillText'] = text
    except Exception as e:
        print(f"Error processing {pdf_url}: {e}")
        df.at[index, 'BillText'] = ''

# Check the dataframe with extracted text
df.head()

Error processing https://webserver.rilegislature.gov/BillText/BillText24/HouseText24/H7626.pdf: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
Error processing https://webserver.rilegislature.gov/BillText/BillText24/HouseText24/H7608.pdf: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
Error processing https://webserver.rilegislature.gov/BillText/BillText24/HouseText24/H7686.pdf: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
Error processing https://webserver.rilegislature.gov/BillText/BillText24/HouseText24/H7671.pdf: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
Error processing https://webserver.rilegislature.gov/BillText/BillText24/HouseText24/H7625.pdf: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
Error processing https://webserver.rilegislature.gov/BillText/BillText24/HouseText24/H7624.pdf: ('Connection aborted.', Conne

KeyboardInterrupt: 

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

# Prepare the data in the required format
train_texts = train_df['BillText'].tolist()
train_summaries = train_df['Summary_StateLink'].tolist()

val_texts = val_df['BillText'].tolist()
val_summaries = val_df['Summary_StateLink'].tolist()


In [ ]:
!pip install torch torchvision torchaudio

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments

# Load the tokenizer and model
model_name = "philschmid/bart-large-cnn-samsum"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# Tokenize the input text
def tokenize_data(texts, summaries, tokenizer, max_length=512):
    inputs = tokenizer(texts, max_length=max_length, truncation=True, padding="max_length", return_tensors="pt")
    labels = tokenizer(summaries, max_length=max_length, truncation=True, padding="max_length", return_tensors="pt").input_ids
    return inputs.input_ids, inputs.attention_mask, labels

train_encodings, train_masks, train_labels = tokenize_data(train_texts, train_summaries, tokenizer)
val_encodings, val_masks, val_labels = tokenize_data(val_texts, val_summaries, tokenizer)

# Prepare the dataset class



In [ ]:
import torch
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments


In [ ]:
class BillSummaryDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, masks, labels):
        self.encodings = encodings
        self.masks = masks
        self.labels = labels

    def __len__(self):
        return len(self.encodings)

    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings[idx],
            'attention_mask': self.masks[idx],
            'labels': self.labels[idx]
        }

train_dataset = BillSummaryDataset(train_encodings, train_masks, train_labels)
val_dataset = BillSummaryDataset(val_encodings, val_masks, val_labels)

# Set up the training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=4,   # batch size per device during training
    per_device_eval_batch_size=4,    # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch"
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

# Fine-tune the model
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.184600,0.763737
2,0.239900,0.777764
3,0.144400,0.839238


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_

TrainOutput(global_step=1014, training_loss=0.21352603855217703, metrics={'train_runtime': 936.7929, 'train_samples_per_second': 4.323, 'train_steps_per_second': 1.082, 'total_flos': 4388386819276800.0, 'train_loss': 0.21352603855217703, 'epoch': 3.0})

In [ ]:
model.save_pretrained('./fine_tuned_model')
tokenizer.save_pretrained('./fine_tuned_model')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/vocab.json',
 './fine_tuned_model/merges.txt',
 './fine_tuned_model/added_tokens.json')

In [ ]:
# If you have a separate test set, load it similarly to the training and validation sets.
# For now, let's use the validation set.

# Tokenize the test/validation data
test_texts = val_df['BillText'].tolist()
test_summaries = val_df['Summary_StateLink'].tolist()

test_encodings, test_masks, test_labels = tokenize_data(test_texts, test_summaries, tokenizer)

# Prepare the test dataset
test_dataset = BillSummaryDataset(test_encodings, test_masks, test_labels)

# Evaluate the model
eval_results = trainer.evaluate(eval_dataset=test_dataset)

# Print evaluation results
print(eval_results)


{'eval_loss': 0.8392381072044373, 'eval_runtime': 9.438, 'eval_samples_per_second': 15.893, 'eval_steps_per_second': 4.026, 'epoch': 3.0}


In [ ]:
import torch

# Check if a GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the selected device
model.to(device)

# Tokenize the sample texts and move them to the device
inputs = tokenizer(sample_texts, max_length=512, truncation=True, padding="max_length", return_tensors="pt").to(device)

# Generate summaries
summary_ids = model.generate(inputs.input_ids, num_beams=4, max_length=150, early_stopping=True)

# Decode the generated summaries
summaries = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]

# Print the original summaries and the generated summaries
for i, (original, generated) in enumerate(zip(test_summaries[:5], summaries)):
    print(f"Original Summary {i+1}: {original}")
    print(f"Generated Summary {i+1}: {generated}")
    print("\n" + "-"*80 + "\n")


NameError: name 'sample_texts' is not defined

In [ ]:
# Specify your desired directory path for saving the fine-tuned model
new_model_path = '/content/drive/MyDrive/DK/Stateside/Model_Finetunig/Finetuned_Model2'  # Replace with your desired path

# Save the model and tokenizer to the new path
model.save_pretrained(new_model_path)
tokenizer.save_pretrained(new_model_path)

# You can also save the training arguments and other related data if needed
trainer.save_model(new_model_path)


In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer

# Load the fine-tuned model and tokenizer from the new path
model = BartForConditionalGeneration.from_pretrained(new_model_path)
tokenizer = BartTokenizer.from_pretrained(new_model_path)


In [ ]:
---

In [ ]:
!pip install graphviz

In [ ]:
from graphviz import Digraph

# Create a new directed graph
flowchart = Digraph(comment='Fine-Tuning Process Flow')

# Add nodes to the flowchart
flowchart.node('A', 'Data Preprocessing\n- Load Dataset\n- Extract PDF text\n- Clean Data\n- Split Data')
flowchart.node('B', 'Tokenization\n- Tokenize Input Texts\n- Tokenize Target Summaries')
flowchart.node('C', 'Model Setup\n- Load Pre-Trained BART Model\n- Move Model to GPU/CPU')
flowchart.node('D', 'Fine-Tuning\n- Define Training Arguments\n- Train the Model\n- Monitor Loss')
flowchart.node('E', 'Evaluation\n- Generate Summaries\n- Compare with Ground Truth')
flowchart.node('F', 'Save Fine-Tuned Model\n- Save Model & Tokenizer\n- Save Training State')
flowchart.node('G', 'Load Model for Inference\n- Load Model & Tokenizer\n- Perform Summarization')

# Add edges (arrows) between nodes
flowchart.edges(['AB', 'BC', 'CD', 'DE', 'EF', 'FG'])

# Render the flowchart to a file and show it
flowchart.render('fine_tuning_process_flow', format='png', cleanup=False)
flowchart.view()


'fine_tuning_process_flow.pdf'